In [1]:
!pip install transformers
from transformers import pipeline

import torch
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.3 MB/s eta 0:00:00


In [2]:
data_dir = '/content/drive/MyDrive/Linguistic abstraction summer 2023/Datasets/Raw text data/'

In [3]:
pilot_nyu = pd.read_csv('/content/pilot_NYU_text_export_192_8.8.22 - pilot_NYU_text_export_192_8.8.22.csv', index_col=0)

In [5]:
pilot_nyu.head(3)

,S1_ellab,S12_ellab,A1_ellab,A12_ellab
ID,,,,
1,I am currently a first-year student at NYU and...,NaN,NaN,Since coming to NYU I have been on campus more...
2,NaN,The social life in NYU is interesting. There a...,"I am majoring Media, Culture, Communication. T...",NaN
3,NaN,My social life so far has been pretty good. I ...,My academic life at NYU has been pretty rigoro...,NaN


In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [25]:
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

In [12]:
doc = pilot_nyu.iloc[0,0]
doc

'I am currently a first-year student at NYU and a commuter student so developing a social life has been rather difficult. However, I became an E-board member of a club where I am able to participate in events and meet other people. Also, since I am a nursing major I take courses similar to others in my field and that has made it easier to make friends.'

In [19]:
summarizer(doc, min_length=1, max_length=50)

[{'summary_text': 'Nina is a first-year student at NYU and a commuter student. She is an E-board member of a club where she is able to participate in events and meet other people.'}]

In [15]:
def generate_pegasus_summary(df, col):
  col_summary = []
  for i in range(df.shape[0]):
    txt = df[col].values[i]
    if type(txt) is float: 
      col_summary.append(float('nan'))
      continue

    summary = summarizer(txt, min_length=0, max_length=len(txt.split())//2+2)
    col_summary.append(summary[0]['summary_text'])
  return col_summary

In [19]:
for col in pilot_nyu.columns:
  col_summary = generate_pegasus_summary(pilot_nyu, col)
  pilot_nyu[col+'_summary_bart'] = col_summary

In [12]:
pilot_nyu

,S1_ellab,S12_ellab,A1_ellab,A12_ellab,S1_ellab_summary_bart,S12_ellab_summary_bart,A1_ellab_summary_bart
ID,,,,,,,
1,I am currently a first-year student at NYU and...,NaN,NaN,Since coming to NYU I have been on campus more...,Nina is a first-year student at NYU and a comm...,NaN,NaN
2,NaN,The social life in NYU is interesting. There a...,"I am majoring Media, Culture, Communication. T...",NaN,NaN,There are,"I am majoring in Media, Culture,"
3,NaN,My social life so far has been pretty good. I ...,My academic life at NYU has been pretty rigoro...,NaN,NaN,My social life so far has been pretty good. I ...,"At first she felt inadequate, but as she conti..."
4,I have meet a lot of friends after entering NY...,NaN,NaN,NYU has the greatest professors in the world. ...,"After entering NYU, I have met a lot of friend...",NaN,NaN
5,NaN,NYU is a very diverse college with many opport...,NYU is a fun college to come to if you are ric...,NaN,NaN,As someone who is Asian and from the middle cl...,NYU is a fun college to come to if you are ric...
...,...,...,...,...,...,...,...
188,My social life is going better than expected. ...,NaN,NaN,"Well, so far my life has been pretty busy! I a...",My social life is going better than last semes...,NaN,NaN
189,NaN,My social life at NYU has been very challengin...,"My academic life at NYU has been challenging, ...",NaN,NaN,At NYU it took a long time for her to form mea...,"At NYU, I am able to take classes that will di..."
190,NaN,My social life might not be what everyone desi...,As a student at NYU it can be hard to manage y...,NaN,NaN,"As long as you know how to balance yourself, t...",As a student at NYU it can be hard to manage y...


In [26]:
txt = 'very boring'
summarizer(txt, min_length=0, max_length=2)

[{'summary_text': ''}]

In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [36]:
for l,r in zip(range(1,5), range(5,9)):
  s = []
  txt_col, summary_col = pilot_nyu.iloc[:,l], pilot_nyu.iloc[:,r]
  for i in range(pilot_nyu.shape[0]):
    txt, summary = txt_col[i], summary_col[i]
    if type(txt) is not float and (summary=='' or type(summary) is float): 
      a.append(' '.join(list(filter(lambda x: x not in stop_words, txt.split()))))
    else: a.append(summary)
  pilot_nyu.iloc[:,r] = s

In [38]:
pilot_nyu.to_csv('/content/drive/MyDrive/pilot_nyu_bart.csv')